In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob

from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
directory = '~/PycharmProjects/tfm_hugopobil'

btc_usd_grouped = pd.read_csv(f'{directory}/data/sampled_data/btc_usd_grouped_v2.csv')
tweets_grouped = pd.read_csv(f'{directory}/data/sampled_data/tweets_grouped_v2.csv')

In [3]:
df_clean = pd.read_csv(f'{directory}/data/sampled_data/tweets_clean_v2.csv')
df_clean.head()

,index,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,sample_date
0,21519,Rahul Chahal,NaN,#Bitcoin #BTC,2009-03-26 10:41:20,100.0,388.0,3401.0,False,2021-02-05 10:53:49,Bitcoin and ETH both have bullish setups for a...,"['Bitcoin', 'ETH', 'BTC']",Twitter for iPad,False,2021-02-05
1,21507,Iconic Funds,"Frankfurt, Germany",Professional Crypto Asset Management\nhttps://...,2017-08-03 10:44:25,16813.0,818.0,1201.0,False,2021-02-05 11:00:24,4⃣ 🎙️ Bloomberg LP CryptoOutlook 2021 with ⬇️...,"['CryptoOutlook', 'cryptocurrency', 'bitcoin',...",Twitter Web App,False,2021-02-05
2,21491,CryptoSquawk,Australia,24x7 Crypto market real-time audio squawk for ...,2017-10-05 10:13:09,1282.0,25.0,72.0,False,2021-02-05 11:14:46,⬇️⬇️ $BTC SELLING PRESSURE ALERT 📉 Price tradi...,"['Bitcoin', 'BTC', 'crypto']",CryptoSquawkBot,False,2021-02-05
3,21483,Drew MacMartin,NaN,"Co-Founder Wealth Playbook, Macro Economics & ...",2016-03-05 15:34:35,73.0,122.0,274.0,False,2021-02-05 11:20:02,"If hyperinflation does hit again, think of the...","['hyperinflation', 'inflation', 'flood', 'GOLD...",Twitter for iPhone,False,2021-02-05
4,21443,DeriBot.info,"Chicago, IL",https://t.co/HsKyGWNVqh - The Fastest Cloud Se...,2013-07-27 07:40:54,1973.0,127.0,631.0,False,2021-02-05 11:45:50,DeriBot Daily Trading Report 5.02.2021 11:42 U...,['Bitcoin'],Twitter Web App,False,2021-02-05


In [4]:
df_clean.dropna(subset=['hashtags'], inplace=True)
df = df_clean[['text']]
df.columns = ['tweets']
df.head()

,tweets
0,Bitcoin and ETH both have bullish setups for a...
1,4⃣ 🎙️ Bloomberg LP CryptoOutlook 2021 with ⬇️...
2,⬇️⬇️ $BTC SELLING PRESSURE ALERT 📉 Price tradi...
3,"If hyperinflation does hit again, think of the..."
4,DeriBot Daily Trading Report 5.02.2021 11:42 U...


In [5]:
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words(['english'])

print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
lem = WordNetLemmatizer()

In [7]:
def cleaning(data):
    tweet_without_url = re.sub(r'http\S+',' ', data)
    tweet_without_hashtag = re.sub(r'#\w+', ' ', tweet_without_url)
    tweet_without_mentions = re.sub(r'@\w+',' ', tweet_without_hashtag)
    precleaned_tweet = re.sub('[^A-Za-z]+', ' ', tweet_without_mentions)

    # Tokenization
    tweet_tokens = TweetTokenizer().tokenize(precleaned_tweet)
    tokens_without_punc = [w for w in tweet_tokens if w.isalpha()]
    tokens_without_sw = [t for t in tokens_without_punc if t not in stop_words]

    text_cleaned = [lem.lemmatize(t) for t in tokens_without_sw]

    return " ".join(text_cleaned)

In [8]:
df['cleaned_tweets'] = df['tweets'].apply(lambda x: cleaning(x))
df['date'] = df_clean['date']
df['date_clean'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df.drop(columns='date',inplace=True)
df.head()

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_9263/2555527066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_tweets'] = df['tweets'].apply(lambda x: cleaning(x))
/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_9263/2555527066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df_clean['date']
/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_9263/2555527066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,tweets,cleaned_tweets,date_clean
0,Bitcoin and ETH both have bullish setups for a...,Bitcoin ETH bullish setup move higher BTC woul...,2021-02-05
1,4⃣ 🎙️ Bloomberg LP CryptoOutlook 2021 with ⬇️...,Bloomberg LP CryptoOutlook cryptocurrency bitc...,2021-02-05
2,⬇️⬇️ $BTC SELLING PRESSURE ALERT 📉 Price tradi...,BTC SELLING PRESSURE ALERT Price trading aroun...,2021-02-05
3,"If hyperinflation does hit again, think of the...",If hyperinflation hit think inflation like flo...,2021-02-05
4,DeriBot Daily Trading Report 5.02.2021 11:42 U...,DeriBot Daily Trading Report UTC Bitcoin Tradi...,2021-02-05


In [9]:
def get_Subjectivity(tweet):
    return TextBlob(tweet).sentiment.subjectivity

def get_Polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

In [10]:
btc_usd_grouped = btc_usd_grouped.set_index('Date')

In [11]:
btc_usd_grouped

,Adj Close
Date,
2021-02-06,39266.011719
2021-02-07,38903.441406
2021-02-08,46196.464844
2021-02-09,46481.105469
2021-02-10,44918.183594
...,...
2022-01-24,36654.328125
2022-01-25,36954.003906
2022-01-26,36852.121094


In [37]:
def btc_price_cate(score):
    if score < 1:
        return 'negative'
    elif score == 1:
        return 'neutral'
    else:
        return 'positive'


def observe_period(period):
    res = btc_usd_grouped['Adj Close'].shift(period)/btc_usd_grouped['Adj Close']
    res = res.apply(btc_price_cate)
    return res

In [39]:
# Explain in report
# 7 is the days bitcoin price if shifted, to we will predict for week returns
time_sentiment = observe_period(7)
time_sentiment.index

Index(['2021-02-06', '2021-02-07', '2021-02-08', '2021-02-09', '2021-02-10',
       '2021-02-11', '2021-02-12', '2021-02-13', '2021-02-14', '2021-02-15',
       ...
       '2022-01-19', '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23',
       '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28'],
      dtype='object', name='Date', length=357)

In [41]:
# CHECK
df['crypto_sentiment'] = df.date_clean.apply(lambda x: time_sentiment[x] if x in time_sentiment else np.nan)

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_9263/2522087232.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['crypto_sentiment'] = df.date_clean.apply(lambda x: time_sentiment[x] if x in time_sentiment else np.nan)


In [44]:
# We are going to drop the days for which we cannot measure the crypto_sentiment with the shift
df_2 = df.copy()
df_2 = df_2.dropna()
df_2.shape, df.shape

((20565, 6), (22120, 6))

In [45]:
df_2['subjectivity'] = df_2['cleaned_tweets'].apply(get_Subjectivity)
df_2['polarity'] = df_2['cleaned_tweets'].apply(get_Polarity)
print(df_2.shape)
df_2.head()

(20565, 6)


,tweets,cleaned_tweets,date_clean,crypto_sentiment,subjectivity,polarity
205,"""Will Institutional Investment Keep Pouring In...",Will Institutional Investment Keep Pouring Int...,2021-02-06,positive,0.0,0.0
206,BTC Bitcoin You know where the WSB money is g...,BTC Bitcoin You know WSB money going WallStree...,2021-02-06,positive,0.0,0.0
208,"🔼🔼 ₿1 = $38,868 (00:56 UTC)\n$BTC prices conti...",UTC BTC price continue rise Change since midni...,2021-02-06,positive,0.0,0.0
209,BTC Bitcoin All the way up! 🚀 🚀 💵 💵 /xVyLbbWRiu,BTC Bitcoin All way xVyLbbWRiu,2021-02-06,positive,0.0,0.0
210,Keep going BTC bitcoin,Keep going BTC bitcoin,2021-02-06,positive,0.0,0.0


In [46]:
def getSentiment(score, threshold=0.5):
    if score < 0:
        return 'negative'
    elif 0.001 < score < threshold:
        return 'neutral'
    else:
        return 'positive'

df_2['sentiment'] = df_2['polarity'].apply(getSentiment)
df_2['target'] = df_2['sentiment'] == df_2['crypto_sentiment']
df_2.head()

,tweets,cleaned_tweets,date_clean,crypto_sentiment,subjectivity,polarity,sentiment,target
205,"""Will Institutional Investment Keep Pouring In...",Will Institutional Investment Keep Pouring Int...,2021-02-06,positive,0.0,0.0,positive,True
206,BTC Bitcoin You know where the WSB money is g...,BTC Bitcoin You know WSB money going WallStree...,2021-02-06,positive,0.0,0.0,positive,True
208,"🔼🔼 ₿1 = $38,868 (00:56 UTC)\n$BTC prices conti...",UTC BTC price continue rise Change since midni...,2021-02-06,positive,0.0,0.0,positive,True
209,BTC Bitcoin All the way up! 🚀 🚀 💵 💵 /xVyLbbWRiu,BTC Bitcoin All way xVyLbbWRiu,2021-02-06,positive,0.0,0.0,positive,True
210,Keep going BTC bitcoin,Keep going BTC bitcoin,2021-02-06,positive,0.0,0.0,positive,True


In [49]:
df_2.target.value_counts(normalize=True)

False    0.679115
True     0.320885
Name: target, dtype: float64

In [50]:
df_2.crypto_sentiment.value_counts()

positive    10534
negative    10031
Name: crypto_sentiment, dtype: int64

In [51]:
df_2.sentiment.value_counts(normalize=True)

positive    0.542086
neutral     0.348796
negative    0.109117
Name: sentiment, dtype: float64

In [52]:
df_2 = df_2.set_index('date_clean')
df_2.head()

,tweets,cleaned_tweets,crypto_sentiment,subjectivity,polarity,sentiment,target
date_clean,,,,,,,
2021-02-06,"""Will Institutional Investment Keep Pouring In...",Will Institutional Investment Keep Pouring Int...,positive,0.0,0.0,positive,True
2021-02-06,BTC Bitcoin You know where the WSB money is g...,BTC Bitcoin You know WSB money going WallStree...,positive,0.0,0.0,positive,True
2021-02-06,"🔼🔼 ₿1 = $38,868 (00:56 UTC)\n$BTC prices conti...",UTC BTC price continue rise Change since midni...,positive,0.0,0.0,positive,True
2021-02-06,BTC Bitcoin All the way up! 🚀 🚀 💵 💵 /xVyLbbWRiu,BTC Bitcoin All way xVyLbbWRiu,positive,0.0,0.0,positive,True
2021-02-06,Keep going BTC bitcoin,Keep going BTC bitcoin,positive,0.0,0.0,positive,True


In [53]:
df_2.to_csv(f'{directory}/data/sampled_data/tweets_nlp_modelling_v3.csv')